In [1]:
import tensorflow as tf
import numpy as np
import google.protobuf
from IPython.display import Audio
import matplotlib.pyplot as plt

import multiprocessing

sess = tf.Session()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def filename_to_example_array(data_filename):
    with multiprocessing.Pool(processes=4) as pool:
        dataset_iterator = tf.python_io.tf_record_iterator(data_filename)
        data = pool.map(tf.train.Example.FromString, dataset_iterator)
    return np.array(data)

In [3]:
def midi_pitch_to_frequency(pitch):
    '''
    pitch: int, a midi pitch
    
    returns the frequency represented by that pitch
    '''
    return np.power(2, (pitch-69)/12) * 440

def frequency_to_midi_pitch(frequency):
    '''
    frequency: float, the frequency of a note
    
    returns the closest midi pitch
    '''
    return 69 + np.round(12 * np.log2(frequency / 440))

def midi_pitch_to_note_name(pitch, flats=True):
    '''
    pitch: int, the midi number of a note
    
    returns the note name and octave
    note that no sanity checking is done on the note,
        negative numbers are interpreted as if the midi range extended that far
    '''
    if flats:
        notes = [
            'A',
            'Bb',
            'B',
            'C',
            'Db',
            'D',
            'Eb',
            'E',
            'F',
            'Gb',
            'G',
            'Ab'
        ]
    else:
        notes = [
            'A',
            'A#',
            'B',
            'C',
            'C#',
            'D',
            'D#',
            'E',
            'F',
            'F#',
            'G',
            'G#'
        ]
    
    # determined from http://newt.phys.unsw.edu.au/jw/notes.html
    
    note = notes[(pitch - 21) % len(notes)]
    octave = int(np.ceil((pitch-23) / len(notes)))
    return note + str(octave)

In [4]:
def example_to_audio(example):
    'example: tf.train.Example containing a note'
    sample_rate = example.features.feature['sample_rate'].int64_list.value[0]
    sample = example.features.feature['audio'].float_list.value
    return Audio(sample, rate=sample_rate)

In [5]:
def get_feature(datum, feature):
    '''
    datum: tf.train.Feature from the magenta dataset
            (should have all the things in feature_conversion_map)
    
    returns the extracted value as int, str, or np.array
    
    TODO: should probably bytes.decode the items in qualities_str?
          haven't tested most of these
    '''
    def from_int64(x): return x.int64_list.value[0]
    def from_bytes(x): return bytes.decode(x.bytes_list.value[0])
    def from_float_list(x): return np.array(x.float_list.value)
    def from_int64_list(x): return np.array(x.int64_list.value)
    def from_bytes_list(x): return np.array(x.bytes_list)
    
    feature_conversion_map = {
        'note' : from_int64,
        'note_str' : from_bytes,
        'instrument' : from_int64,
        'instrument_str' : from_bytes,
        'pitch' : from_int64,
        'velocity' : from_int64,
        'sample_rate' : from_int64,
        'audio' : from_float_list,
        'qualities' : from_int64_list,
        'qualities_str' : from_bytes_list,
        'instrument_family' : from_int64,
        'instrument_family_str' : from_bytes,
        'instrument_source' : from_int64,
        'instrument_source_str' : from_bytes
    }
    
    return feature_conversion_map[feature](datum.features.feature[feature])
    

In [6]:
def merge_audio(datum_1, datum_2):
    '''
    datum_1, datum_2: tf.train.Feature from the magenta dataset
            or anything with 'audio' and 'sample_rate' fields
        sample_rate fields should be the same
    
    returns tf.train.Feature with audio, sample_rate features
    
    TODO add the rest, some combination of the other stuff
    '''
    sample_rate_1 = get_feature(datum_1, 'sample_rate')
    sample_rate_2 = get_feature(datum_2, 'sample_rate')
    assert (sample_rate_1 == sample_rate_2)
    
    audio_1 = get_feature(datum_1, 'audio')
    audio_2 = get_feature(datum_2, 'audio')
    
    audio = (audio_1 + audio_2) / 2
    audio_feature = tf.train.Feature(float_list=tf.train.FloatList(value=audio))
    sample_rate_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[sample_rate_1]))
    
    features = {
        'audio' : audio_feature,
        'sample_rate' : sample_rate_feature,
    }
    example = tf.train.Example(features=tf.train.Features(feature=features))
    return example

In [7]:
data_filename = './Data/nsynth-test.tfrecord'

data = filename_to_example_array(data_filename)

In [8]:
idx = np.random.choice(range(len(data)))

In [9]:
datum = data[idx]
print('{}: {:s}'.format(idx, get_feature(datum, 'note_str')))
print(midi_pitch_to_note_name(get_feature(datum, 'pitch')))
example_to_audio(datum)

2503: flute_acoustic_002-099-127
Eb7


In [10]:
# 1800: organ_electronic A4
# 4075: reed_acoustic C5
datum_1 = data[1800]
datum_2 = data[4075]
datum_3 = merge_audio(datum_1, datum_2)
example_to_audio(datum_3)